In [154]:
# imports
import pandas as pd
import numpy as np
import gradio as gr

In [155]:
file_name = "data/googleplaystore.csv"
df = pd.read_csv(file_name, encoding="utf-8")


In [156]:
# เข้าใจข้อมูล
# check dataset info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


In [157]:
# check null
df.isna().sum()
# จะเห็นได้ว่า rating มีข้อมูลเสียเยอะ

App                  0
Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       1
Genres               0
Last Updated         0
Current Ver          8
Android Ver          3
dtype: int64

In [158]:
# data prep & data cleaning
# เราจะใช้ข้อมูลที่จำเป็นสำหรับการใช้ข้อมูลนี้เท่านั้น
# ข้อมูลที่ไม่จำเป็นเราจะนำออกไป เช่น rating, reviews บลาๆ

df_temp = df.copy();

# remove unuse cols
df_temp = df.reindex(columns=['App', 'Category', 'Type', 'Genres', 'Price', 'Size', 'Installs'])

In [159]:
# rename col
temp_keys = {}
for i in df_temp.columns:
    temp_keys[i] = i.lower()
df_temp = df_temp.rename(columns=temp_keys)

In [160]:
df_temp.isna().sum()
# ข้อมูลที่เสียมีน้อยมาก

app         0
category    0
type        1
genres      0
price       0
size        0
installs    0
dtype: int64

In [161]:
# แสดงข้อมูลเบื้องต้น
df_temp.head(10)

,app,category,type,genres,price,size,installs
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,Free,Art & Design,0,19M,"10,000+"
1,Coloring book moana,ART_AND_DESIGN,Free,Art & Design;Pretend Play,0,14M,"500,000+"
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,Free,Art & Design,0,8.7M,"5,000,000+"
3,Sketch - Draw & Paint,ART_AND_DESIGN,Free,Art & Design,0,25M,"50,000,000+"
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,Free,Art & Design;Creativity,0,2.8M,"100,000+"
5,Paper flowers instructions,ART_AND_DESIGN,Free,Art & Design,0,5.6M,"50,000+"
6,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,Free,Art & Design,0,19M,"50,000+"
7,Infinite Painter,ART_AND_DESIGN,Free,Art & Design,0,29M,"1,000,000+"
8,Garden Coloring Book,ART_AND_DESIGN,Free,Art & Design,0,33M,"1,000,000+"
9,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,Free,Art & Design;Creativity,0,3.1M,"10,000+"


In [162]:
# เช็คข้อมูล insight หลักๆ ที่อาจจะทำให้เรามีปัญหาในการใช้ข้อมูล
df_temp.groupby(['category'])['category'].count()

category
1.9                       1
ART_AND_DESIGN           65
AUTO_AND_VEHICLES        85
BEAUTY                   53
BOOKS_AND_REFERENCE     231
BUSINESS                460
COMICS                   60
COMMUNICATION           387
DATING                  234
EDUCATION               156
ENTERTAINMENT           149
EVENTS                   64
FAMILY                 1972
FINANCE                 366
FOOD_AND_DRINK          127
GAME                   1144
HEALTH_AND_FITNESS      341
HOUSE_AND_HOME           88
LIBRARIES_AND_DEMO       85
LIFESTYLE               382
MAPS_AND_NAVIGATION     137
MEDICAL                 463
NEWS_AND_MAGAZINES      283
PARENTING                60
PERSONALIZATION         392
PHOTOGRAPHY             335
PRODUCTIVITY            424
SHOPPING                260
SOCIAL                  295
SPORTS                  384
TOOLS                   843
TRAVEL_AND_LOCAL        258
VIDEO_PLAYERS           175
WEATHER                  82
Name: category, dtype: int64

In [163]:
# จะเห็นได้ว่า 1.9 นั้นไม่รู้คือ type อะไร
# เรามาลองแก้กันดีกว่า
df_temp.loc[df_temp['category'] == '1.9']

,app,category,type,genres,price,size,installs
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,0,"February 11, 2018",Everyone,"1,000+",Free


In [164]:
# ประเภทที่ใกล้เคียงที่สุดกับประเภทด้านบนที่สุด คงจะเป็น HOUSE_AND_HOME
# เพราะเป็นของที่ใช้ตกแต่งบ้าน และต้องใช้แอพในการเปลี่ยนรูป
# เราเลยจะเปลี่ยนข้อมูลให้กับข้อมูลนี้

df_temp['category'] = df_temp['category'].replace(["1.9"], 'HOUSE_AND_HOME')
df_temp.loc[df_temp['category'] == '1.9']

# ข้อมูลได้หายไปแล้ว


,app,category,type,genres,price,size,installs


In [165]:
# เพื่อความแน่ใจ ลองเช็คข้อมูลอีกรอบ
df_temp.loc[df_temp['app'] == 'Life Made WI-Fi Touchscreen Photo Frame']

# ข้อมูลได้ถูกเปลี่ยนแล้วเย้

,app,category,type,genres,price,size,installs
10472,Life Made WI-Fi Touchscreen Photo Frame,HOUSE_AND_HOME,0,"February 11, 2018",Everyone,"1,000+",Free


In [166]:
# data cleaning
# จะเห็นได้ว่า install เป็น type object ต้องเปลี่ยนเป็น int
# จะเห็นได้ว่า size เป็น type object ต้องเปลี่ยนเป็น int
# type จะมีสองรูปแบบคือ free กับ paid ต้องเปลี่ยนเป็น bin ถึงจะได้ง่ายต่อการทำงาน